In [1]:
#pip install scikit-image --upgrade-strategy only-if-needed
import numpy as np
import pandas as pd
import os

import cv2
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import dask_image.imread
import dask

In [2]:
#import
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
import os
import json 
import random
from xgboost import XGBClassifier
from tensorflow.keras.applications.resnet import ResNet50
import tensorflow as tf
from tensorflow import keras
from sklearn import metrics
from torch.utils.data import Dataset
import torch
path = os.getcwd()

In [3]:
from dask import delayed, compute
import dask

In [4]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
print(cluster,client)
# client.close()

LocalCluster(28f79ad0, 'tcp://127.0.0.1:60417', workers=4, threads=16, memory=31.93 GiB) <Client: 'tcp://127.0.0.1:60417' processes=4 threads=16, memory=31.93 GiB>


In [ ]:
import xgboost
xgboost.__version__

In [6]:
# client.close()

In [6]:
# 제공된 sample data는 파프리카와 시설포도 2종류의 작물만 존재
# 변수 설명 csv 파일 참조
crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'1':'초기','2':'중기','3':'말기'}

label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

global label_encoder
label_encoder = {key:idx for idx, key in enumerate(label_description)}
label_decoder = {val:key for key, val in label_encoder.items()}



@delayed
def label_encoding(label):
    global label_encoder
    encoded_label = label_encoder[label]
    return encoded_label
    

In [7]:
class Controller():
    def __init__(self,csv_feature_dict,len_of_df,feat_len_of_df):
        
        self.csv_feature_dict = csv_feature_dict
        self.len_of_df = len_of_df
        self.feat_len_of_df = feat_len_of_df
    
    @delayed
    def get_csv(self,csv_path):
        
        df = pd.read_csv(csv_path)[self.csv_feature_dict.keys()]
        df = df.replace('-', 0)
        
        return df

    @delayed
    def scaling(self,df,):
        
        for col in df.columns:
            df[col] = df[col].astype(float) - self.csv_feature_dict[col][0]
            df[col] = df[col] / (self.csv_feature_dict[col][1]-self.csv_feature_dict[col][0])
        return df
    @delayed
    def padding(self,df):
        
        pad = np.zeros((self.len_of_df, len(df.columns)))
        length = min(self.len_of_df, len(df))
        pad[-length:] = df.to_numpy()[-length:]
        csv = pad.reshape(-1,self.len_of_df,self.feat_len_of_df)
        return csv
        
    @delayed
    def get_all_array(self,csv_path_list):
        
        csvarr = np.empty((0,self.len_of_df,self.feat_len_of_df), float)
        for ind,csv_path in enumerate(csv_path_list):
                # csv = get_csv(csv_path,csv_feature_dict).compute()
                # csv = scaling(csv,csv_feature_dict).compute()
                # csv = padding(csv,len_of_df).compute().reshape(-1,len_of_df,feat_len_of_df)
                csv = self.get_csv(csv_path)
                csv = self.scaling(csv,)
                csv = self.padding(csv,)
                csvarr = delayed(np.append)(csvarr,csv, axis = 0)
        return csvarr
    
     
    @delayed
    def getimage(self,imgpath,image_size):
        img = cv2.imread(imgpath)
        img = cv2.resize(img, dsize=(image_size, image_size), interpolation=cv2.INTER_AREA)
        img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
        # img = np.transpose(img, (2,0,1))
        return img.reshape(-1,image_size,image_size,3)
    
    @delayed
    def get_all_image(self,imgpath_list,image_size):
        imgarr = np.empty((0,image_size,image_size,3), float)
        for ind,img_path in enumerate(imgpath_list):
            img = self.getimage(img_path,image_size)
            imgarr = delayed(np.append)(imgarr,img, axis = 0)
        return imgarr
    
    
    @delayed
    def getlable(self,jsonpath):
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)

        crop = json_file['annotations']['crop']
        disease = json_file['annotations']['disease']
        risk = json_file['annotations']['risk']
        label = f'{crop}_{disease}_{risk}'
        return label
    
    
    @delayed
    def get_label_list(self,labelpath_list):
        labelarr = np.array([])
        for ind,json_path in enumerate(labelpath_list):
            label = label_encoding(self.getlable(json_path))
            labelarr = np.append(labelarr,label)
        return labelarr
            
            
            

In [8]:
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

# 분석에 사용할 feature 선택
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

csv_files = sorted(glob(path + '\\data\\train\\*\\*.csv'))

temp_csv = pd.read_csv(csv_files[0])[csv_features]
max_arr, min_arr = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()

# feature 별 최대값, 최솟값 계산
for csv in tqdm(csv_files[1:]):
    temp_csv = pd.read_csv(csv)[csv_features]
    temp_csv = temp_csv.replace('-',np.nan).dropna()#.astype(float)
    if len(temp_csv) == 0:
        continue
    temp_csv = temp_csv.astype(float)
    temp_max, temp_min = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()
    max_arr = np.max([max_arr,temp_max], axis=0)
    min_arr = np.min([min_arr,temp_min], axis=0)

# feature 별 최대값, 최솟값 dictionary 생성
csv_feature_dict = {csv_features[i]:[min_arr[i], max_arr[i]] for i in range(len(csv_features))}

100%|██████████| 5759/5759 [00:44<00:00, 128.22it/s]


In [9]:
#트레인셋경로
train_csv_pathlist = sorted(glob(path + '\\data\\train\\*\\*.csv'))
train_img_list = sorted(glob(path + '\\data\\train\\*\\*.jpg'))
train_label_list = sorted(glob(path + '\\data\\train\\*\\*.json'))
#테스트셋경로
test_csv_pathlist = sorted(glob(path + '\\data\\test\\*\\*.csv'))
test_img_list = sorted(glob(path + '\\data\\test\\*\\*.jpg'))

#CSV 파일셋 설정
csvfeature_len = 9
csvrows_len = 144
#IMG 파일셋 설정
img_size = 224

In [100]:
###############트레인##############
# trainbatch = 100
# #csv 파일셋 로드
# controller_train = Controller(train_csv_pathlist[:trainbatch],csv_feature_dict,csvrows_len,csvfeature_len)
# train_x1 = controller_train.get_all_array()
# #IMG 파일셋 로드
# train_x2 = controller_train.get_all_image(train_img_list[:trainbatch],img_size)
# #label 파일셋 로드
# y_train = controller_train.get_label_list(train_label_list[:trainbatch])

In [89]:
###############테스트##############
# #CSV 파일셋 로드
# csvcontroller_test = Controller(test_csv_pathlist,csv_feature_dict,csvrows_len,csvfeature_len)
# test_x1 = csvcontroller_test.get_all_array()
# #IMG 파일셋 로드
# test_x2 = controller_train.get_all_image(test_img_list,img_size)


In [10]:
class Eencoder(keras.layers.Layer):
    global max_len
    def __init__(self):
        super(Eencoder, self).__init__()
        self.block1_layer1 = tf.keras.layers.Conv1D(9, 24*2, activation='relu',)#input_shape=input_shape[1:])
        self.block1_layer2 = tf.keras.layers.Conv1D(18, 24*2+1, activation='relu',)#input_shape=input_shape[1:])
        self.block1_layer3 = tf.keras.layers.Conv1D(36, 24*2+1, activation='relu',)#input_shape=input_shape[1:])
         
    def call(self, inputs):
        #LSTM파트
        lstm_x = self.block1_layer1(inputs)
        lstm_x = tf.nn.relu(lstm_x)
        lstm_x = self.block1_layer2(lstm_x)
        lstm_x = tf.nn.relu(lstm_x)
        lstm_x = self.block1_layer3(lstm_x)
        lstm_x = tf.nn.relu(lstm_x)
        
        return lstm_x
class Ddecoder(keras.layers.Layer):
    def __init__(self):
        super(Ddecoder, self).__init__()
        self.block1_layer1 = tf.keras.layers.Conv1DTranspose(3,  24*2, activation='relu',)#input_shape=input_shape[1:])
        self.block1_layer2 = tf.keras.layers.Conv1DTranspose(6,  24*2+1, activation='relu',)#input_shape=input_shape[1:])
        self.block1_layer3 = tf.keras.layers.Conv1DTranspose(9,  24*2+1, activation='relu',)#input_shape=input_shape[1:])
        
         
    def call(self, inputs):
        #LSTM파트
        x = self.block1_layer1(inputs)
        
        x = tf.nn.relu(x)
        x = self.block1_layer2(x)
        
        x = tf.nn.relu(x)
        x = self.block1_layer3(x)
       
        return x

# Eencoder()(x_train[0][:2,:,:]).shape
# Ddecoder()(Eencoder()(x_train[0][:2,:,:]))

class Autoencoder(tf.keras.Model): 
  def __init__(self,): 
    super(Autoencoder, self).__init__() 
    self.encoder = Eencoder() 
    self.decoder = Ddecoder() 
  
  def call(self, input): 
    code = self.encoder(input) 
    reconstructed = self.decoder(code) 
    return reconstructed

def loss(model, original): 
  reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(model(original), original))) 
  return reconstruction_error

def train(loss, model, opt, original): 
  with tf.GradientTape() as tape: 
    gradients = tape.gradient(loss(model, original), model.trainable_variables) 
    gradient_variables = zip(gradients, model.trainable_variables) 
    opt.apply_gradients(gradient_variables)



In [11]:
minibatch_size = 300
data_length = len(train_csv_pathlist)
loop_num = int(np.floor(data_length/minibatch_size))
controller = Controller(csv_feature_dict,csvrows_len,csvfeature_len)

automodel = Autoencoder()
opt = tf.optimizers.Adam()
loss_fn = keras.losses.MeanSquaredError()
automodel.compile(optimizer=opt, loss=loss_fn)
# automodel.fit(train_x1, train_x1, 
#                 )
count = 0
for i in range(loop_num):
    
    if i != loop_num-1:
        #csv 파일셋 로드
        train_x1 = controller.get_all_array(train_csv_pathlist[count*minibatch_size:(count+1)*minibatch_size])
        #IMG 파일셋 로드
        # train_x2 = controller.get_all_image(train_img_list[count*minibatch_size:(count+1)*minibatch_size],img_size)
        # #label 파일셋 로드
        # y_train = controller.get_label_list(train_label_list[count*minibatch_size:(count+1)*minibatch_size])
        
        automodel.fit(train_x1.compute().compute(),train_x1.compute().compute())
    else:
        train_x1 = controller.get_all_array(train_csv_pathlist[count*minibatch_size:])
        automodel.fit(train_x1.compute().compute(),train_x1.compute().compute())
    
    count +=1
    

12/12 [==============================] - 0s 9ms/step - loss: 0.0203


In [13]:
class preprmodel(keras.Model):
    def __init__(self, imgmodel,autoencoder,name = None):
        super(preprmodel, self).__init__()
        self.imgmodel = imgmodel
        self.autoencoder = autoencoder

    
    def call(self, inputs):
        x1 = self.imgmodel(inputs[1])
        x2 = self.autoencoder.encoder(inputs[0]).numpy().reshape(-1,36)


        x = tf.concat([x1,x2],axis=1)
        return(x)
    
model_RESNET50 = ResNet50(weights='imagenet')
prepromodel = preprmodel(model_RESNET50,automodel)
# x_prime_train = prepromodel(x_train)
# x_prime_test = prepromodel(x_test)

In [24]:
def return_y(y):
    return y.compute()


minibatch_size = 200
data_length = len(train_csv_pathlist)
loop_num = int(np.floor(data_length/minibatch_size))

xgb_1 = XGBClassifier(random_state=100, subsample= 0.7, colsample_bytree=0.7, scale_pos_weight=0.2)
# for i 
# xgb.fit(x_prime_train,y_train,xgb_model = xgb)
count = 0
for i in range(loop_num):
# for i in range(1):
    
    xgb_1 = XGBClassifier(random_state=100, subsample= 0.7, colsample_bytree=0.7, scale_pos_weight=0.2,xgb_model=xgb_1)
    print(i)
    if i != loop_num-1:
        #csv 파일셋 로드
        train_x1 = controller.get_all_array(train_csv_pathlist[count*minibatch_size:(count+1)*minibatch_size])
        #IMG 파일셋 로드
        train_x2 = controller.get_all_image(train_img_list[count*minibatch_size:(count+1)*minibatch_size],img_size)
        # #label 파일셋 로드
        y_train = controller.get_label_list(train_label_list[count*minibatch_size:(count+1)*minibatch_size])
        
        xgb_1.fit(prepromodel([train_x1.compute().compute(),train_x2.compute().compute()]),list(map(return_y, y_train.compute())))
    else:
        #csv 파일셋 로드
        train_x1 = controller.get_all_array(train_csv_pathlist[count*minibatch_size:])
        #IMG 파일셋 로드
        train_x2 = controller.get_all_image(train_img_list[count*minibatch_size:],img_size)
        # #label 파일셋 로드
        y_train = controller.get_label_list(train_label_list[count*minibatch_size:])
        xgb_1.fit(prepromodel([train_x1.compute().compute(),train_x2.compute().compute()]),list(map(return_y, y_train.compute())))
        
    
    count +=1

0


C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:56:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "scale_pos_weight", "xgb_model" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:56:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1
[23:57:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "scale_pos_weight", "xgb_model" } might not be used.

  This could be a false alarm, with some parameters getting used by language bi

# 예측

In [ ]:
submission = pd.read_csv(path + '\\data\\sample_submission.csv')
# submission['label'] = preds
minibatch_size = 200
data_length = len(test_csv_pathlist)
loop_num = int(np.floor(data_length/minibatch_size))

# for i 
# xgb.fit(x_prime_train,y_train,xgb_model = xgb)
count = 0
for i in range(loop_num):
# for i in range(2):
    print(i)
    if i != loop_num-1:
        start_p = count*minibatch_size
        end_p = (count+1)*minibatch_size
        #csv 파일셋 로드
        test_x1 = controller.get_all_array(test_csv_pathlist[start_p:end_p])
        #IMG 파일셋 로드
        test_x2 = controller.get_all_image(test_img_list[start_p:end_p],img_size)
        
        #예측
        pred_raw = xgb_1.predict(prepromodel([test_x1.compute().compute(),test_x2.compute().compute()]))
        submission.loc[start_p:end_p-1,'label'] = np.array([label_decoder[int(val)] for val in pred_raw])
        
    
    count += 1

0
1


In [ ]:
submission

,image,label
0,10000,6_00_0
1,10001,5_b6_1
2,10002,3_b7_1
3,10003,3_00_0
4,10004,5_b8_1
...,...,...
51901,67673,0_00_0
51902,67674,0_00_0
51903,67675,0_00_0
51904,67676,0_00_0


In [ ]:
submission.to_csv('gogo_submission.csv', index=False)